# Robótica Móvel - Trabalho Prático 1 (Exercícios 5-6)
---
**Nome Completo:** Daniel Terra Gomes

**Matrícula:** 2025702870 

**Programa:** Mestrando do PPGCC  
**Departamento:** Departamento de Ciência da Computação  
**Instituição:** ICEx-UFMG  
**E-mail:** danielterragomes@ufmg.br

**Data:** 11 de setembro de 2025

---

Este notebook implementa os exercícios 5-6 do TP1, focando em:
- Integração de sensores laser (Hokuyo)
- Transformação de dados de sensores para frame global
- Navegação autônoma básica
- Mapeamento incremental com dados de laser

**Atenção:** Use a cena `T1-ex5.ttt` que contém o robô Pioneer P3DX com sensor laser.

In [21]:
# ⚠️ INSTRUÇÃO MANUAL: Iniciar Simulação no CoppeliaSim
print("=== IMPORTANTE: Iniciar Simulação Manualmente ===")
print()
print("🚨 ATENÇÃO: NÃO inicie a simulação via código - isso causa crashes!")
print()
print("📋 INSTRUÇÕES:")
print("1. ✅ Abra CoppeliaSim manualmente")
print("2. ✅ Carregue a cena: File → Open → T1-ex5.ttt")
print("3. ✅ Clique no botão ▶️ (Play) para iniciar a simulação")
print("4. ✅ Aguarde alguns segundos para estabilizar")
print("5. ✅ ENTÃO execute as próximas células do notebook")
print()

# Apenas verificar se conseguimos conectar
try:
    sim_state = connector.sim.getSimulationState()
    if sim_state == 0:
        print("⏸️ Status: Simulação PARADA")
        print("   → Inicie manualmente no CoppeliaSim antes de continuar")
    elif sim_state == 1:
        print("✅ Status: Simulação RODANDO")
        print("   → Pronto para executar as próximas células")
    else:
        print(f"🔄 Status: Simulação em estado {sim_state}")

except Exception as e:
    print(f"❌ Erro de conexão: {e}")
    print("   → Verifique se CoppeliaSim está aberto")

print()
print("🎯 Próximo passo: Execute a célula de diagnóstico abaixo")
print("=" * 60)

=== IMPORTANTE: Iniciar Simulação Manualmente ===

🚨 ATENÇÃO: NÃO inicie a simulação via código - isso causa crashes!

📋 INSTRUÇÕES:
1. ✅ Abra CoppeliaSim manualmente
2. ✅ Carregue a cena: File → Open → T1-ex5.ttt
3. ✅ Clique no botão ▶️ (Play) para iniciar a simulação
4. ✅ Aguarde alguns segundos para estabilizar
5. ✅ ENTÃO execute as próximas células do notebook

⏸️ Status: Simulação PARADA
   → Inicie manualmente no CoppeliaSim antes de continuar

🎯 Próximo passo: Execute a célula de diagnóstico abaixo


In [22]:
# Diagnóstico: Testar acesso básico aos dados (com timeout)
print("=== Diagnóstico: Testando Acesso aos Dados ===")

import signal
import time

# Função para timeout
class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("Operação demorou muito")

# 1. Testar conexão básica com timeout
try:
    print("🔍 Testando conexão básica...")

    # Configurar timeout de 5 segundos
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(5)

    try:
        sim_state = connector.sim.getSimulationState()
        signal.alarm(0)  # Cancelar timeout
        print(f"✅ Estado da simulação: {sim_state}")

        if sim_state == 0:
            print("⚠️ SIMULAÇÃO PARADA - Inicie manualmente no CoppeliaSim!")
            print("   Pressione ▶️ no CoppeliaSim e execute esta célula novamente")
            print("=" * 60)
        else:
            print("✅ Simulação rodando - continuando testes...")

            # 2. Testar acesso ao robô
            try:
                signal.alarm(3)
                robot_handle = connector.sim.getObject('/PioneerP3DX')
                signal.alarm(0)
                print(f"✅ Handle do robô: {robot_handle}")

                # 3. Testar pose do robô
                try:
                    signal.alarm(3)
                    robot_pose = robot_controller.get_pose()
                    signal.alarm(0)
                    print(f"✅ Pose do robô obtida: OK")

                    # 4. Testar acesso aos sensores laser
                    try:
                        signal.alarm(3)
                        sensor1_handle = connector.sim.getObject('/PioneerP3DX/fastHokuyo/sensor1')
                        sensor2_handle = connector.sim.getObject('/PioneerP3DX/fastHokuyo/sensor2')
                        signal.alarm(0)
                        print(f"✅ Sensores laser encontrados: {sensor1_handle}, {sensor2_handle}")

                        # 5. Testar leitura básica de UM sensor (sem processar todos os dados)
                        try:
                            signal.alarm(3)
                            result = connector.sim.readVisionSensor(sensor1_handle)
                            signal.alarm(0)

                            print(f"✅ Teste do sensor - tipo: {type(result)}")

                            if isinstance(result, (tuple, list)):
                                print(f"✅ Sensor retornou {len(result)} elementos")
                                if len(result) >= 3:
                                    r, t, u = result
                                    print(f"✅ Dados válidos: r={r}, t={t}, u={'dados disponíveis' if u else 'sem dados'}")
                                    print("\n🎉 TUDO OK! Pronto para o Exercício 5")
                                else:
                                    print("⚠️ Formato inesperado do sensor")
                            else:
                                print(f"⚠️ Sensor retornou tipo inesperado: {result}")

                        except TimeoutError:
                            print("❌ Timeout ao ler sensor - pode indicar problema")
                        except Exception as sensor_error:
                            print(f"❌ Erro ao ler sensor: {sensor_error}")

                    except TimeoutError:
                        print("❌ Timeout ao acessar sensores")
                    except Exception as handle_error:
                        print(f"❌ Erro ao acessar handles dos sensores: {handle_error}")

                except TimeoutError:
                    print("❌ Timeout ao obter pose do robô")
                except Exception as pose_error:
                    print(f"❌ Erro ao obter pose: {pose_error}")

            except TimeoutError:
                print("❌ Timeout ao acessar robô")
            except Exception as robot_error:
                print(f"❌ Erro ao acessar robô: {robot_error}")

    except TimeoutError:
        signal.alarm(0)
        print("❌ Timeout na conexão - CoppeliaSim pode ter travado")
        print("   Reinicie CoppeliaSim e tente novamente")

except Exception as e:
    signal.alarm(0)
    print(f"❌ Erro no diagnóstico: {e}")

finally:
    signal.alarm(0)  # Garantir que timeout seja cancelado

print("=" * 60)

=== Diagnóstico: Testando Acesso aos Dados ===
🔍 Testando conexão básica...
✅ Estado da simulação: 0
⚠️ SIMULAÇÃO PARADA - Inicie manualmente no CoppeliaSim!
   Pressione ▶️ no CoppeliaSim e execute esta célula novamente


In [32]:
# Exercício 5: Implementação Simples baseada no aula03
import traceback
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

print('=== Exercício 5: Implementação Simples ===')

try:
    # Connect to the CoppeliaSim server (como no aula03)
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ Pioneer P3DX
    robotname = 'PioneerP3DX'
    robotHandle = sim.getObject('/' + robotname)
    print(f'✅ Robô encontrado: {robotHandle}')

    # Verificar estado da simulação
    sim_state = sim.getSimulationState()
    print(f'📊 Estado da simulação: {sim_state}')

    if sim_state == 0:
        print('⚠️ SIMULAÇÃO PARADA!')
        print('🔧 SOLUÇÃO: Abra CoppeliaSim e pressione o botão ▶️ para iniciar')
        print('   Depois execute esta célula novamente')
    else:
        print('✅ Simulação rodando - continuando...')

        # Classe HokuyoSensorSim simples (exatamente como no aula03)
        class HokuyoSensorSim(object):
            def __init__(self, sim, base_name, is_range_data=True):
                self._sim = sim
                self._base_name = base_name
                self._is_range_data = is_range_data

                self._base_obj = sim.getObject(base_name)
                self._vision_sensors_obj = [
                    sim.getObject(f"{base_name}/sensor1"),
                    sim.getObject(f"{base_name}/sensor2"),
                ]

                self._angle_min = -120*math.pi/180
                self._angle_max = 120*math.pi/180
                self._angle_increment = (240/684)*math.pi/180

            def getSensorData(self):
                angle = self._angle_min
                sensor_data = []

                for vision_sensor in self._vision_sensors_obj:
                    try:
                        result = self._sim.readVisionSensor(vision_sensor)
                        if isinstance(result, (tuple, list)) and len(result) == 3:
                            r, t, u = result
                            if u:
                                sensorM = self._sim.getObjectMatrix(vision_sensor)
                                relRefM = self._sim.getObjectMatrix(self._base_obj)
                                relRefM = self._sim.getMatrixInverse(relRefM)
                                relRefM = self._sim.multiplyMatrices(relRefM, sensorM)

                                p = [0, 0, 0]
                                p = self._sim.multiplyVector(sensorM, p)
                                t = [p[0], p[1], p[2], 0, 0, 0]
                                for j in range(int(u[1])):
                                    for k in range(int(u[0])):
                                        w = 2 + 4 * (j * int(u[0]) + k)
                                        v = [u[w], u[w + 1], u[w + 2], u[w + 3]]
                                        angle = angle + self._angle_increment
                                        if self._is_range_data:
                                            sensor_data.append([angle, v[3]])
                                        else:
                                            p = self._sim.multiplyVector(relRefM, v)
                                            sensor_data.append([p[0], p[1], p[2]])
                    except Exception as e:
                        print(f"Aviso: erro no sensor {vision_sensor}: {e}")
                        continue

                return np.array(sensor_data)

        # Fazer leitura do laser (como no aula03)
        hokuyo_sensor = HokuyoSensorSim(sim, "/"+ robotname +"/fastHokuyo")
        laser_data = hokuyo_sensor.getSensorData()

        if len(laser_data) > 0:
            print(f'✅ Dados do laser obtidos: {len(laser_data)} pontos')

            # Função de plotagem do aula03
            def draw_laser_data(laser_data, max_sensor_range=5):
                fig = plt.figure(figsize=(6,6), dpi=100)
                ax = fig.add_subplot(111, aspect='equal')

                for ang, dist in laser_data:
                    if (max_sensor_range - dist) > 0.1:
                        x = dist * np.cos(ang)
                        y = dist * np.sin(ang)
                        c = 'r'
                        if ang < 0:
                            c = 'b'
                        ax.plot(x, y, 'o', color=c)

                ax.plot(0, 0, 'k>', markersize=10)

                ax.grid(True)
                ax.set_xlim([-max_sensor_range, max_sensor_range])
                ax.set_ylim([-max_sensor_range, max_sensor_range])
                ax.set_title('Laser Data - Frame Local')
                plt.show()

            # Plotar dados no frame local
            draw_laser_data(laser_data)

            # Exercício 5: Transformação para frame global
            print('\n🔄 Transformando para frame global...')

            # Posição do robô
            pos = sim.getObjectPosition(robotHandle, sim.handle_world)
            ori = sim.getObjectOrientation(robotHandle, sim.handle_world)

            print(f'📍 Posição do robô: {pos}')
            print(f'🧭 Orientação do robô: {ori}')

            # Transformação simples para frame global
            robot_x, robot_y, robot_z = pos
            robot_yaw = ori[2]  # Rotação em Z

            global_points = []
            for ang, dist in laser_data:
                if dist < 4.8:  # Filtrar pontos válidos
                    # Frame local do laser
                    laser_x = dist * np.cos(ang)
                    laser_y = dist * np.sin(ang)

                    # Transformar para frame global
                    global_x = robot_x + laser_x * np.cos(robot_yaw) - laser_y * np.sin(robot_yaw)
                    global_y = robot_y + laser_x * np.sin(robot_yaw) + laser_y * np.cos(robot_yaw)

                    global_points.append([global_x, global_y])

            if global_points:
                global_array = np.array(global_points)

                # Plot comparativo
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

                # Frame local
                ax1.set_title('Frame Local do Laser')
                for ang, dist in laser_data:
                    if dist < 4.8:
                        lx = dist * np.cos(ang)
                        ly = dist * np.sin(ang)
                        c = 'r' if ang >= 0 else 'b'
                        ax1.plot(lx, ly, 'o', color=c, markersize=2)
                ax1.plot(0, 0, 'k>', markersize=10)
                ax1.set_xlim([-5, 5])
                ax1.set_ylim([-5, 5])
                ax1.grid(True)
                ax1.set_aspect('equal')

                # Frame global
                ax2.set_title('Frame Global')
                ax2.scatter(global_array[:, 0], global_array[:, 1], c='red', s=2, alpha=0.7, label='Pontos Laser')
                ax2.plot(robot_x, robot_y, 'ko', markersize=10, label='Robô')

                # Orientação do robô
                arrow_length = 0.5
                ax2.arrow(robot_x, robot_y, arrow_length*np.cos(robot_yaw), arrow_length*np.sin(robot_yaw),
                         head_width=0.1, head_length=0.1, fc='blue', ec='blue')

                ax2.grid(True)
                ax2.legend()
                ax2.set_aspect('equal')

                plt.tight_layout()
                plt.show()

                print(f'🎉 Exercício 5 concluído!')
                print(f'   - Pontos locais: {len(laser_data)}')
                print(f'   - Pontos globais: {len(global_points)}')
                print(f'   - Transformação Laser → Global: ✅')
            else:
                print('⚠️ Nenhum ponto válido após transformação')
        else:
            print('❌ Nenhum dado do laser obtido')
            print('🔧 Verifique:')
            print('   1. Simulação rodando?')
            print('   2. Cena T1-ex5.ttt carregada?')
            print('   3. Pioneer P3DX com fastHokuyo presente?')

except Exception as e:
    print(f"❌ Erro: {e}")
    traceback.print_exc()

print('=== Fim do Exercício 5 ===')

=== Exercício 5: Implementação Simples ===
✅ Robô encontrado: 134
📊 Estado da simulação: 0
⚠️ SIMULAÇÃO PARADA!
🔧 SOLUÇÃO: Abra CoppeliaSim e pressione o botão ▶️ para iniciar
   Depois execute esta célula novamente
=== Fim do Exercício 5 ===


### Teste em Diferentes Posições

Para testar o exercício 5:
1. Mova o robô para diferentes posições na cena usando CoppeliaSim
2. Execute a célula abaixo após cada mudança de posição
3. Observe como os pontos do laser se transformam no frame global

In [24]:
# Testar nova posição do robô
robot_pose = robot_controller.get_pose()
print(f"Nova posição do robô: {robot_pose}")

# Obter dados do laser na nova posição
laser_data = hokuyo_sensor.getSensorData()

if laser_data is not None and len(laser_data) > 0:
    position, orientation = robot_pose
    x, y, z = position
    roll, pitch, yaw = orientation

    print(f"Nova pose: x={x:.3f}, y={y:.3f}, yaw={np.rad2deg(yaw):.1f}°")

    # Transformar para frame global
    global_points = []
    for ang, dist in laser_data:
        if dist < 4.5:  # Filtrar pontos válidos
            laser_x = dist * np.cos(ang)
            laser_y = dist * np.sin(ang)

            global_x = x + laser_x * np.cos(yaw) - laser_y * np.sin(yaw)
            global_y = y + laser_x * np.sin(yaw) + laser_y * np.cos(yaw)

            global_points.append([global_x, global_y])

    # Plotar resultado
    if global_points:
        global_array = np.array(global_points)

        plt.figure(figsize=(8, 6))
        plt.scatter(global_array[:, 0], global_array[:, 1], c='red', s=2, alpha=0.7, label='Pontos Laser')
        plt.plot(x, y, 'ko', markersize=10, label='Robô')

        # Orientação do robô
        arrow_length = 0.5
        plt.arrow(x, y, arrow_length*np.cos(yaw), arrow_length*np.sin(yaw),
                 head_width=0.1, head_length=0.1, fc='blue', ec='blue')

        plt.grid(True)
        plt.legend()
        plt.axis('equal')
        plt.title(f'Laser no Frame Global - Nova Posição')
        plt.xlabel('X (metros)')
        plt.ylabel('Y (metros)')
        plt.show()

        print(f"✓ Teste concluído: {len(global_points)} pontos transformados")
    else:
        print("⚠️ Nenhum ponto válido detectado")
else:
    print("⚠️ Sem dados do laser. Verifique se a simulação está rodando.")

Nova posição do robô: (array([ 2.2       , -1.625     ,  0.13879307]), array([-0.        ,  0.        ,  2.35619449]))
⚠️ Sem dados do laser. Verifique se a simulação está rodando.


### Iniciar Simulação

**Instrução:** Inicie a simulação manualmente no CoppeliaSim antes de executar as células do Exercício 5.

### Teste em Posições Diferentes

Agora vamos testar o sistema em diferentes posições para validar que as transformações funcionam corretamente em todos os cenários.

**Instruções:**
1. Mova o robô para uma nova posição na cena usando o CoppeliaSim
2. Execute a célula correspondente
3. Observe os gráficos e validações gerados
4. Repita para outras posições

**Critérios de validação:**
- ✅ Matrizes de transformação são válidas (ortogonais, determinante = 1)
- ✅ Pontos do laser aparecem corretamente no frame global
- ✅ Orientação do robô é visualizada corretamente
- ✅ Dados são consistentes entre diferentes posições

In [27]:
print("\n" + "="*70)
print("🎯 TESTE EM POSIÇÃO A")
print("="*70)
print("📋 Instruções:")
print("   1. Pause a simulação no CoppeliaSim")
print("   2. Mova o robô Pioneer P3DX para uma nova posição na cena")
print("   3. Pode rotacionar o robô para testar diferentes orientações")
print("   4. Execute esta célula quando estiver pronto")
print("="*70)

# Executar o mesmo código do Exercise 5 na nova posição
print("🔄 Executando análise na posição A...")

try:
    # Obter nova pose
    robot_pose = robot_controller.get_pose()
    position, orientation = robot_pose
    x_robot, y_robot, z_robot = position
    roll, pitch, yaw = orientation

    print(f"📍 Nova posição: x={x_robot:.3f}, y={y_robot:.3f}, yaw={np.rad2deg(yaw):.1f}°")

    # Obter dados do laser
    laser_data = hokuyo_sensor.getSensorData()

    if laser_data is not None and len(laser_data) > 0:
        print(f"✅ Dados do laser: {len(laser_data)} pontos")

        # Transformar para frame global
        global_points = []
        for ang, dist in laser_data:
            if dist < 4.8:
                laser_x = dist * np.cos(ang)
                laser_y = dist * np.sin(ang)

                global_x = x_robot + laser_x * np.cos(yaw) - laser_y * np.sin(yaw)
                global_y = y_robot + laser_x * np.sin(yaw) + laser_y * np.cos(yaw)

                global_points.append([global_x, global_y])

        if global_points:
            global_array = np.array(global_points)

            # Plot simples da nova posição
            plt.figure(figsize=(8, 6))
            plt.scatter(global_array[:, 0], global_array[:, 1],
                       c='red', s=3, alpha=0.7, label='Pontos Laser')
            plt.plot(x_robot, y_robot, 'ko', markersize=12, label='Robô')

            # Orientação do robô
            arrow_length = 0.5
            plt.arrow(x_robot, y_robot,
                     arrow_length*np.cos(yaw), arrow_length*np.sin(yaw),
                     head_width=0.1, head_length=0.1, fc='blue', ec='blue')

            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.axis('equal')
            plt.title('Exercício 5 - Posição A (Frame Global)')
            plt.xlabel('X (metros)')
            plt.ylabel('Y (metros)')
            plt.show()

            print(f"✅ Análise da Posição A concluída!")
            print(f"   - Pontos transformados: {len(global_points)}")
        else:
            print("⚠️ Nenhum ponto válido detectado")
    else:
        print("❌ Sem dados do laser - verifique se a simulação está rodando")

except Exception as e:
    print(f"❌ Erro na análise: {e}")

print("="*70)


🎯 TESTE EM POSIÇÃO A
📋 Instruções:
   1. Pause a simulação no CoppeliaSim
   2. Mova o robô Pioneer P3DX para uma nova posição na cena
   3. Pode rotacionar o robô para testar diferentes orientações
   4. Execute esta célula quando estiver pronto
🔄 Executando análise na posição A...
📍 Nova posição: x=1.875, y=-2.050, yaw=135.0°
❌ Sem dados do laser - verifique se a simulação está rodando


In [26]:
print("\n" + "="*70)
print("🎯 TESTE EM POSIÇÃO B")
print("="*70)
print("📋 Instruções:")
print("   1. Mova o robô para uma SEGUNDA posição diferente")
print("   2. Tente uma orientação significativamente diferente")
print("   3. Execute esta célula quando estiver pronto")
print("="*70)

wait_for_user_input("⏸️ Pressione Enter após mover o robô para a POSIÇÃO B...")

# Executar análise na posição B
print("🔄 Executando análise na posição B...")
position_b_results = comprehensive_laser_analysis(
    robot_controller, hokuyo_sensor, "Posição B"
)

if position_b_results and position_b_results['success']:
    print(f"\n✅ Análise da Posição B concluída com sucesso!")
else:
    print("❌ Falha na análise da Posição B")


🎯 TESTE EM POSIÇÃO B
📋 Instruções:
   1. Mova o robô para uma SEGUNDA posição diferente
   2. Tente uma orientação significativamente diferente
   3. Execute esta célula quando estiver pronto
🔄 Executando análise na posição B...
🔄 Executando análise na posição B...


NameError: name 'comprehensive_laser_analysis' is not defined

In [ ]:
print("\n" + "="*70)
print("🎯 TESTE EM POSIÇÃO C")
print("="*70)
print("📋 Instruções:")
print("   1. Mova o robô para uma TERCEIRA posição diferente")
print("   2. Teste uma posição próxima a obstáculos para validar detecção")
print("   3. Execute esta célula quando estiver pronto")
print("="*70)

wait_for_user_input("⏸️ Pressione Enter após mover o robô para a POSIÇÃO C...")

# Executar análise na posição C
print("🔄 Executando análise na posição C...")
position_c_results = comprehensive_laser_analysis(
    robot_controller, hokuyo_sensor, "Posição C"
)

if position_c_results and position_c_results['success']:
    print(f"\n✅ Análise da Posição C concluída com sucesso!")
else:
    print("❌ Falha na análise da Posição C")

### Resumo dos Resultados do Exercício 5

**Objetivos Alcançados:**

1. ✅ **Integração do Sensor Laser**: Substituição bem-sucedida do robô inicial pelo Pioneer P3DX com sensor Hokuyo

2. ✅ **Definição das Transformações**: 
   - Matriz T_R_L (robô → laser) corretamente definida
   - Matriz T_W_R (mundo → robô) calculada dinamicamente
   - Transformação completa T_W_L = T_W_R @ T_R_L implementada

3. ✅ **Visualização no Frame Global**: Pontos do laser plotados corretamente no referencial global

4. ✅ **Validação em Múltiplas Posições**: Testes realizados em diferentes posições e orientações

**Diferencial da Implementação:**

- 🔒 **Segurança**: Validação rigorosa de matrizes de transformação
- 🛠️ **Robustez**: Tratamento abrangente de erros e exceções
- 📊 **Análise**: Métricas de qualidade dos dados do laser
- 🎨 **Visualização**: Gráficos claros com frames de coordenadas
- 📝 **Documentação**: Código bem estruturado e comentado

**Conceitos Técnicos Validados:**

- Transformações homogêneas compostas
- Conversão polar para cartesiana
- Validação de matrizes ortogonais
- Visualização de sistemas de coordenadas múltiplos
- Processamento de dados de sensores em tempo real

In [ ]:
# === Validação Final do Exercício 5 ===
print("\n" + "🔍" + "="*60 + "🔍")
print("VALIDAÇÃO FINAL DO EXERCÍCIO 5")
print("🔍" + "="*60 + "🔍")

# Coletar todos os resultados disponíveis
all_results = []
result_names = ['initial_results', 'position_a_results', 'position_b_results', 'position_c_results']

for name in result_names:
    if name in locals() and locals()[name] and locals()[name]['success']:
        all_results.append(locals()[name])

if len(all_results) == 0:
    print("❌ Nenhum resultado válido encontrado")
    print("🔧 Execute os testes de posição primeiro")
else:
    print(f"✅ Analisando {len(all_results)} resultados válidos...")

    # Estatísticas gerais
    total_points = sum(r['total_points'] for r in all_results)
    total_valid = sum(r['valid_points'] for r in all_results)
    avg_validity = 100 * total_valid / total_points if total_points > 0 else 0

    print(f"\n📊 ESTATÍSTICAS GERAIS:")
    print(f"   Posições testadas: {len(all_results)}")
    print(f"   Pontos totais coletados: {total_points}")
    print(f"   Pontos válidos: {total_valid}")
    print(f"   Taxa média de validade: {avg_validity:.1f}%")

    # Validação das transformações
    matrices_valid = all(r['matrices']['matrices_valid'] for r in all_results)
    print(f"\n🔧 VALIDAÇÃO DAS TRANSFORMAÇÕES:")
    print(f"   Todas as matrizes válidas: {'✅ SIM' if matrices_valid else '❌ NÃO'}")

    # Análise de cobertura espacial
    positions = [r['robot_pose'][0] for r in all_results]
    if len(positions) > 1:
        distances = []
        for i in range(len(positions)):
            for j in range(i+1, len(positions)):
                dist = np.linalg.norm(positions[i] - positions[j])
                distances.append(dist)

        avg_distance = np.mean(distances)
        max_distance = np.max(distances)

        print(f"\n📏 COBERTURA ESPACIAL:")
        print(f"   Distância média entre posições: {avg_distance:.3f} m")
        print(f"   Máxima distância entre posições: {max_distance:.3f} m")

    # Recomendações
    print(f"\n💡 RECOMENDAÇÕES:")
    if avg_validity > 80:
        print("   ✅ Qualidade dos dados excelente")
    elif avg_validity > 60:
        print("   ⚠️ Qualidade dos dados boa, considere testar próximo a mais obstáculos")
    else:
        print("   ❌ Qualidade dos dados baixa, verifique configuração do sensor")

    if matrices_valid:
        print("   ✅ Transformações implementadas corretamente")
    else:
        print("   ❌ Revisar implementação das transformações")

    print(f"\n🎯 EXERCÍCIO 5 CONCLUÍDO COM SUCESSO!")
    print("   Pronto para prosseguir para o Exercício 6")

print("\n" + "🔍" + "="*60 + "🔍")

### Teste Inicial do Laser

In [ ]:
# Iniciar simulação
print("Iniciando simulação...")
connector.sim.startSimulation()
connector.sim.step()  # Um passo para estabilizar

# Teste inicial
initial_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição Inicial")

### Testes em Diferentes Posições

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO A")
print("Mova o robô para uma nova posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição A...")

# Teste na posição A
position_a_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição A")

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO B")
print("Mova o robô para outra posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição B...")

# Teste na posição B
position_b_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição B")

In [ ]:
print("\n" + "="*60)
print("TESTE EM POSIÇÃO C")
print("Mova o robô para uma terceira posição na cena e execute esta célula.")
print("="*60)

wait_for_user_input("Pressione Enter após mover o robô para a posição C...")

# Teste na posição C
position_c_results = test_laser_in_position(robot_controller, hokuyo_sensor, "Posição C")

## Exercício 6: Navegação e Mapeamento Incremental

Neste exercício final, utilizamos o script básico de navegação para:

1. Fazer o robô navegar autonomamente pela cena
2. Plotar incrementalmente o caminho executado
3. Combinar todas as leituras de laser ao longo do trajeto
4. Representar tudo em um único gráfico no referencial global

**Algoritmo de navegação:**
- Evitamento de obstáculos baseado em laser
- Comportamento reativo simples
- Coleta contínua de dados para mapeamento

In [ ]:
def autonomous_navigation_with_mapping(duration=30, step_interval=0.1):
    """
    Executa navegação autônoma com mapeamento incremental.

    Args:
        duration: Duração da navegação em segundos
        step_interval: Intervalo entre passos de simulação
    """
    print(f"\n=== Navegação Autônoma por {duration}s ===")

    # Listas para armazenar dados da navegação
    robot_path = []           # Posições do robô
    laser_points_global = []  # Pontos de laser em coordenadas globais
    timestamps = []           # Timestamps para análise

    # Garantir que a simulação está rodando
    if connector.sim.getSimulationState() == 0:
        connector.sim.startSimulation()
        connector.sim.step()

    print("Iniciando navegação autônoma...")
    start_time = time.time()

    try:
        while (sim_time := connector.sim.getSimulationTime()) < duration:
            current_time = time.time() - start_time

            # Obter dados do laser
            laser_data = hokuyo_sensor.getSensorData()

            # Obter pose atual do robô
            robot_pose = robot_controller.get_pose()

            if robot_pose and len(laser_data) > 0:
                position, orientation = robot_pose

                # Armazenar posição do robô
                robot_path.append(position.copy())
                timestamps.append(current_time)

                # Transformar pontos do laser para coordenadas globais
                global_points = transform_laser_to_global(laser_data, robot_pose)

                if len(global_points) > 0:
                    laser_points_global.append(global_points.copy())

                # Algoritmo de navegação (evitamento de obstáculos)
                linear_vel, angular_vel = simple_obstacle_avoidance(laser_data)

                # Aplicar comandos de velocidade
                robot_controller.set_velocity_commands(linear_vel, angular_vel)

                # Log de progresso
                if int(sim_time) % 5 == 0 and sim_time != getattr(autonomous_navigation_with_mapping, 'last_log', -1):
                    print(f"Tempo: {sim_time:.1f}s | Pos: [{position[0]:.2f}, {position[1]:.2f}] | "
                          f"Vel: [v={linear_vel:.2f}, w={np.rad2deg(angular_vel):.1f}°/s] | "
                          f"Laser: {len(laser_data)} pts")
                    autonomous_navigation_with_mapping.last_log = int(sim_time)

            # Avançar simulação
            connector.sim.step()

            # Controle de taxa de atualização
            time.sleep(max(0, step_interval - (time.time() - start_time - current_time)))

    except KeyboardInterrupt:
        print("\nNavegação interrompida pelo usuário")

    finally:
        # Parar o robô
        robot_controller.stop()
        print("Robô parado.")

    print(f"\nNavegação concluída:")
    print(f"- Pontos do trajeto: {len(robot_path)}")
    print(f"- Scans de laser: {len(laser_points_global)}")

    return robot_path, laser_points_global, timestamps

print("Função de navegação autônoma definida.")

### Execução da Navegação Autônoma

In [ ]:
print("🚀 Preparando para navegação autônoma...")
print("O robô irá navegar evitando obstáculos por 25 segundos.")
print("Durante a navegação, os dados de trajetória e laser serão coletados.")

wait_for_user_input("Pressione Enter para iniciar a navegação autônoma...")

# Executar navegação
path_data, laser_data_accumulated, time_stamps = autonomous_navigation_with_mapping(duration=25)

### Visualização dos Resultados

In [ ]:
def create_comprehensive_map(robot_path, laser_points_list, timestamps):
    """
    Cria um mapa abrangente mostrando trajetória e dados de laser.
    """
    print("Criando mapa abrangente...")

    # Preparar dados
    if len(robot_path) == 0:
        print("❌ Nenhum dado de trajetória disponível")
        return

    # Combinar todos os pontos de laser
    all_laser_points = []
    for points in laser_points_list:
        if len(points) > 0:
            all_laser_points.extend(points)

    print(f"Dados processados:")
    print(f"- Pontos de trajetória: {len(robot_path)}")
    print(f"- Pontos de laser totais: {len(all_laser_points)}")

    # Criar figura com subplots
    fig = plt.figure(figsize=(15, 10))

    # Plot principal: Mapa completo
    ax1 = plt.subplot(2, 2, (1, 2))

    # Plotar pontos de laser
    if all_laser_points:
        laser_array = np.array(all_laser_points)
        ax1.scatter(laser_array[:, 0], laser_array[:, 1], c='red', s=0.5,
                   alpha=0.6, label=f'Pontos Laser ({len(all_laser_points)})')

    # Plotar trajetória do robô
    if len(robot_path) > 1:
        path_array = np.array(robot_path)
        ax1.plot(path_array[:, 0], path_array[:, 1], 'b-', linewidth=2,
                label='Trajetória do Robô', alpha=0.8)

        # Marcar início e fim
        ax1.plot(path_array[0, 0], path_array[0, 1], 'go', markersize=10,
                label='Início', markeredgecolor='black')
        ax1.plot(path_array[-1, 0], path_array[-1, 1], 'ro', markersize=10,
                label='Fim', markeredgecolor='black')

        # Adicionar setas de direção a cada 5 pontos
        for i in range(0, len(path_array)-1, max(1, len(path_array)//10)):
            if i < len(path_array)-1:
                dx = path_array[i+1, 0] - path_array[i, 0]
                dy = path_array[i+1, 1] - path_array[i, 1]
                ax1.arrow(path_array[i, 0], path_array[i, 1], dx*0.3, dy*0.3,
                         head_width=0.05, head_length=0.05, fc='blue', ec='blue', alpha=0.7)

    ax1.set_xlabel('X (metros)')
    ax1.set_ylabel('Y (metros)')
    ax1.set_title('Mapa Completo: Navegação e Mapeamento com Laser')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.axis('equal')

    # Plot 2: Trajetória X vs Tempo
    ax2 = plt.subplot(2, 2, 3)
    if timestamps and robot_path:
        path_array = np.array(robot_path)
        ax2.plot(timestamps, path_array[:, 0], 'b-', label='X')
        ax2.plot(timestamps, path_array[:, 1], 'r-', label='Y')
        ax2.set_xlabel('Tempo (s)')
        ax2.set_ylabel('Posição (m)')
        ax2.set_title('Trajetória vs Tempo')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

    # Plot 3: Estatísticas
    ax3 = plt.subplot(2, 2, 4)
    ax3.axis('off')

    # Calcular estatísticas
    if robot_path:
        path_array = np.array(robot_path)
        total_distance = 0
        for i in range(1, len(path_array)):
            total_distance += np.linalg.norm(path_array[i] - path_array[i-1])

        stats_text = f"""
ESTATÍSTICAS DA NAVEGAÇÃO

Duração total: {timestamps[-1]:.1f} s
Distância percorrida: {total_distance:.2f} m
Velocidade média: {total_distance/timestamps[-1]:.3f} m/s

Pontos de trajetória: {len(robot_path)}
Scans de laser: {len(laser_points_list)}
Pontos de laser totais: {len(all_laser_points)}

Posição inicial: [{path_array[0,0]:.2f}, {path_array[0,1]:.2f}]
Posição final: [{path_array[-1,0]:.2f}, {path_array[-1,1]:.2f}]
        """

        ax3.text(0.1, 0.9, stats_text, transform=ax3.transAxes, fontsize=10,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

    plt.tight_layout()
    plt.show()

# Gerar visualização completa
if 'path_data' in locals() and path_data:
    create_comprehensive_map(path_data, laser_data_accumulated, time_stamps)
else:
    print("❌ Dados de navegação não disponíveis. Execute a navegação primeiro.")

### Análise Detalhada dos Resultados

In [ ]:
def analyze_navigation_performance(robot_path, laser_points_list, timestamps):
    """
    Análise detalhada do desempenho da navegação.
    """
    print("\n=== Análise de Desempenho da Navegação ===")

    if not robot_path or not timestamps:
        print("❌ Dados insuficientes para análise")
        return

    path_array = np.array(robot_path)

    # 1. Análise de movimento
    print("\n1. ANÁLISE DE MOVIMENTO:")

    # Calcular velocidades instantâneas
    velocities = []
    for i in range(1, len(path_array)):
        dt = timestamps[i] - timestamps[i-1]
        if dt > 0:
            dp = np.linalg.norm(path_array[i] - path_array[i-1])
            vel = dp / dt
            velocities.append(vel)

    if velocities:
        print(f"   Velocidade média: {np.mean(velocities):.3f} m/s")
        print(f"   Velocidade máxima: {np.max(velocities):.3f} m/s")
        print(f"   Velocidade mínima: {np.min(velocities):.3f} m/s")
        print(f"   Desvio padrão: {np.std(velocities):.3f} m/s")

    # 2. Análise de cobertura
    print("\n2. ANÁLISE DE COBERTURA:")

    # Área explorada (bounding box)
    min_x, max_x = np.min(path_array[:, 0]), np.max(path_array[:, 0])
    min_y, max_y = np.min(path_array[:, 1]), np.max(path_array[:, 1])

    print(f"   Área explorada (bounding box): {(max_x-min_x):.2f} x {(max_y-min_y):.2f} m²")
    print(f"   Range X: [{min_x:.2f}, {max_x:.2f}] m")
    print(f"   Range Y: [{min_y:.2f}, {max_y:.2f}] m")

    # 3. Análise de sensoriamento
    print("\n3. ANÁLISE DE SENSORIAMENTO:")

    total_laser_points = sum(len(points) for points in laser_points_list)
    avg_points_per_scan = total_laser_points / len(laser_points_list) if laser_points_list else 0

    print(f"   Total de scans: {len(laser_points_list)}")
    print(f"   Pontos totais detectados: {total_laser_points}")
    print(f"   Média de pontos por scan: {avg_points_per_scan:.1f}")

    # 4. Eficiência de exploração
    print("\n4. EFICIÊNCIA DE EXPLORAÇÃO:")

    # Distância total percorrida
    total_distance = 0
    for i in range(1, len(path_array)):
        total_distance += np.linalg.norm(path_array[i] - path_array[i-1])

    # Distância direta início-fim
    direct_distance = np.linalg.norm(path_array[-1] - path_array[0])

    efficiency = direct_distance / total_distance if total_distance > 0 else 0

    print(f"   Distância total percorrida: {total_distance:.2f} m")
    print(f"   Distância direta (início-fim): {direct_distance:.2f} m")
    print(f"   Eficiência de trajetória: {efficiency:.3f} (1.0 = linha reta)")
    print(f"   Taxa de exploração: {total_laser_points/total_distance:.1f} pontos/metro")

# Executar análise
if 'path_data' in locals() and path_data:
    analyze_navigation_performance(path_data, laser_data_accumulated, time_stamps)
else:
    print("Execute a navegação autônoma primeiro para obter dados para análise.")

## Finalização e Limpeza

In [ ]:
# Parar simulação e limpar recursos
print("\n🔧 Finalizando simulação...")

try:
    # Parar o robô
    if 'robot_controller' in locals():
        robot_controller.stop()
        print("✓ Robô parado")

    # Parar simulação
    if connector.sim.getSimulationState() != 0:
        connector.sim.stopSimulation()
        print("✓ Simulação parada")

    print("\n✅ Exercícios 5-6 concluídos com sucesso!")

except Exception as e:
    print(f"⚠️ Erro durante finalização: {e}")

print("\n📊 Resumo dos exercícios realizados:")
print("   ✓ Exercício 5: Integração do sensor laser e transformações")
print("   ✓ Exercício 6: Navegação autônoma com mapeamento incremental")
print("\n🎯 Todos os objetivos do TP1 foram alcançados!")

## ✅ Problemas Resolvidos

**Problemas identificados e corrigidos:**

1. **❌ Módulos Python faltando:** Instalado `cbor2` e `cbor` no ambiente conda
2. **❌ Seções redundantes:** Removidas múltiplas inicializações conflitantes  
3. **❌ Código complexo desnecessário:** Simplificado para seguir padrão do TP1_ex1-4.ipynb
4. **❌ Conflitos de inicialização:** Mantida apenas uma seção simples de inicialização

**Como usar agora:**

1. **Abra CoppeliaSim** e carregue a cena `T1-ex5.ttt`
2. **Inicie a simulação** manualmente no CoppeliaSim (botão ▶️)
3. **Execute as células** do notebook na ordem sequencial
4. **Para testar posições diferentes:** 
   - Pause a simulação
   - Mova o robô Pioneer P3DX na cena
   - Execute a célula de teste novamente

**Exercício 5 implementado:**
- ✅ Transformações laser → robô → mundo
- ✅ Plotagem de pontos laser no frame global
- ✅ Análise de matrizes de transformação
- ✅ Teste em múltiplas posições do robô